<div class="alert alert-block alert-success">

# <center>  </center>
# <center> Amazon Appliances' Reviews </center>
### <center> Extract Reviews and Metadata</center>
#### <center> JSON/CSV files</center>

**Author: Atef Bader, PhD**
    
**Last Edit: 2/15/2024**  

</div>

In [1]:
import json
import ast
from pandas import DataFrame, Series      


In [2]:

fr=open("meta_Appliances.json")
fw=open("Amazon_Appliances_Metadata.json", "w")

for line in fr:
    json_dat = json.dumps(ast.literal_eval(line))
    dict_dat = json.loads(json_dat)
    json.dump(dict_dat, fw)
    fw.write("\n")

fw.close()
fr.close()


In [3]:

fr=open("reviews_Appliances.json")
fw=open("Amazon_Appliances_Reviews.json", "w")

for line in fr:
    json_dat = json.dumps(ast.literal_eval(line))
    dict_dat = json.loads(json_dat)
    json.dump(dict_dat, fw)
    fw.write("\n")

fw.close()
fr.close()


In [4]:

list_of_appliances_metadata_dict_data = [json.loads(line) for line in open('Amazon_Appliances_Metadata.json')]

In [5]:
appliances_metadata_df = DataFrame(list_of_appliances_metadata_dict_data)

In [ ]:
appliances_metadata_df.head()

In [ ]:
# Inspecting the fields
appliances_metadata_df.info()

In [ ]:
# Questions, Comments, Concerns: 

# One of the things that stands out to me is the data types. The object data type is flexible in that it can store different types
# of data like strings, integers, etc. 

# The columns do not match in their non-null values, meaning several fields will contain multiple instances of NaN. This could potentially pose 
# a problem.

In [ ]:
appliances_metadata_df['related']

In [ ]:
# This field presents some interesting relationships to the review itself.

# Maybe it could be of use in the Group C (Milvus) hybird search query. For example the query given in the overview: “It could 
# so nearly have been a great fridge, but it's the design that brings it down. The doors are clunky, and the buttons on the 
# outside are unappealing and cheap looking" 

# There's a possibility to use a query like this and the 'related' field to see if that customer bought or viewed a similar product based off the
# sentiment in their review for the original product if it was insufficient to their needs. 


In [6]:
list_of_appliances_reviews_dict_data = [json.loads(line) for line in open('Amazon_Appliances_Reviews.json')]

In [7]:
appliances_reviews_df = DataFrame(list_of_appliances_reviews_dict_data)

In [ ]:
appliances_reviews_df.head()

In [ ]:
appliances_reviews_df['unixReviewTime']

In [ ]:
# Questions, Comments, Concerns:

# There is obviously a relationship between 'reviewText' and 'summary', but I'm not certain how 'unixReviewTime' is helpful.

In [ ]:
# Inspecting the fields
appliances_reviews_df.info()

In [ ]:
# All of the fields but 'reviewName' have the same Non-null count. There might be several anonymous reviewers who left their name out of the review
# this shows up as a null value. 

# I do have concerns in regards to the 'asin' field, in the first table there are 11656 entries, but in the reviews table there are 143685 entries. 

# The project overview states that the asin is the ID of each product. When looking at both tables, the data type is an object. Why do the product
# ID's differ in their specific data types? Shouldn't each ID be the same data type to ensure consistency/efficiency? 

In [8]:
appliances_reviews_df.to_csv('Amazon_Appliances_Reviews.csv', encoding='utf-8', index=False)

In [9]:
appliances_metadata_df.to_csv('Amazon_Appliances_Metadata.csv', encoding='utf-8', index=False)

## Requirement 1
### Create a table that has the list of queries for the group that you selected from the groups (Group A, Group B, Group C) listed in the Final Project - Requirements Specification PDF, and add to it the 3 queries that you created in Phase 1 that belong to the group you selected

| Group | Query | Database 
| --- | --- | ---| 
| A | How many products are there in every product subcategory? | Postgres | 
| A | How many product subcategories are in the provided datasets? | Postgres |
| A | What is the average review rating of all reviews? | Postgres |
| A | How many unique reviewers? | Postgres |
| A | How many reviews for every product subcategory | Postgres |
| A | What is the average size (characters/words) of the review text/body? | Postgres |
| A | How many reviews submitted every January for years: 2011, 2012, 2013, 2014? | Postgres |
| A | How many reviews submitted for every product subcategory every May for years: 2011, 2012, 2013, 2014? | Postgres |
| A | What is the average price of every product subcategory that has an average overall rating that is greater than 3? | Postgres |
| A | What are the top 3 reviewed products names in every product subcategory? | Postgres |
| A | How many products have a rating of less than 3? | Postgres |
| A | What is the average number of helpful votes on each review? | Postgres |
| A | What is the average price for every product with a perfect rating of 5? | Postgres |

## Requirement 2
###  Which database have you selected to implement the queries above? What were your benchmark metrics for the database selected?

#### The database I've chosen to run the queries above is the Postgres database. This database is excellent for Group A (Quantitative) queries. The benchmarks for choosing Postgres were as follows:

- PostgreSQL offers robust support for relational data
- PostgreSQL is great for aggregations and conditional filtering on complex queries
- Offers wide variety of primitive data types such as INTEGER, TEXT, VARCHAR, DATE, NUMERIC, etc.
- Known for its reliability, scalability, and advanced features like foreign keys, joins, and stored procedures.

## Requirement 3
### Using the database you selected above, design and develop a database application (insert the data into the database) to host Amazon datasets (Reviews and Metadata).

In [70]:
import psycopg2
import pandas as pd

# Database connection parameters
host = "localhost"       # Your database host
dbname = "amazon_reviews"  # Your database name
user = "postgres"   # Your database username
password = "sql"  # Your database password

# Connect to the PostgreSQL server
try:
    connection = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password
    )
    print("Connection to PostgreSQL database successful")
except Exception as error:
    print(f"Error: Could not connect to the database. {error}")


Connection to PostgreSQL database successful


In [71]:
# Merging dataframes
amazon_merged_df = pd.merge(appliances_reviews_df, appliances_metadata_df, on='asin')


In [59]:
amazon_merged_df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,price,imUrl,description,categories,title,brand,related,salesRank
0,A27HOWZBUBJ8FF,0970408285,Steve,"[0, 0]",Could have been longer though. well made and e...,4.0,NaN,1387152000,"12 16, 2013",40.75,http://ecx.images-amazon.com/images/I/41kQjPIF...,High-pressure hose 90 end & hose washer instal...,"[[Appliances, Parts & Accessories, Washer Part...",NaN,NaN,NaN,NaN
1,A24HQ894NFSTF5,7301113188,"Maha Saqfalhait ""shopaholic! ;)""","[0, 0]",I like these containers so much i have ordered...,5.0,NaN,1236902400,"03 13, 2009",NaN,http://ecx.images-amazon.com/images/I/41EfFOPw...,NaN,"[[Appliances, Freezers, Chest Freezers]]",NaN,NaN,NaN,NaN
2,AXE83MK90ZEVZ,B00002N7HY,Strom,"[0, 0]","It works, no fires, etc. Why not 5 stars? Ho...",4.0,NaN,1389052800,"01 7, 2014",2.29,http://ecx.images-amazon.com/images/I/316Y1KbT...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 5050 50 Amp, 125/250 Volt, NEMA 10-50R...",Leviton,"{'also_bought': ['B000BPFZHM', 'B000VU1KC6', '...",NaN
3,A2J7X7ZIH2EWB1,B00002NATH,NaN,"[0, 0]",Fast shipping. Works great,5.0,NaN,1405814400,"07 20, 2014",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
4,AJQFNOFTZ7GOX,B00002NATH,Barthbill,"[1, 1]",What can I say? It is the usual Leviton high q...,5.0,NaN,1277164800,"06 22, 2010",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
5,A3EIP4ZCGM0U67,B00002NATH,Christian Fischer,"[0, 0]",The Leviton was new in the box. All screws we...,5.0,NaN,1378339200,"09 5, 2013",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
6,A18N8ND2CH5ZZB,B00002NATH,Donn Dorminey,"[0, 0]","When you need one, you need one! But if you ca...",5.0,NaN,1396396800,"04 2, 2014",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
7,A35U08WK3LZ58G,B00002NATH,jim wells,"[0, 0]",i needed some way to plug in my camper and thi...,5.0,NaN,1381622400,"10 13, 2013",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
8,A2PMRZGUQQ6V6Q,B00002NATH,John H,"[3, 3]",Receptacle is of good quality and should work ...,5.0,NaN,1289606400,"11 13, 2010",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN
9,A1BFPCIRWQ3WB9,B00002NATH,John Page,"[0, 0]",The unit was of a high quality and was exactly...,5.0,NaN,1394928000,"03 16, 2014",9.13,http://ecx.images-amazon.com/images/I/41srC2QF...,"Built of durable thermoplastic, Leviton Power ...","[[Appliances, Parts & Accessories, Range Parts...","Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,"{'also_bought': ['B002OUMU66', 'B007HFT034', '...",NaN


In [ ]:
print(amazon_merged_df.dtypes)

In [72]:
amazon_merged_df['summary'] = pd.to_numeric(amazon_merged_df['summary'], errors='coerce')

In [68]:
amazon_merged_df['overall'].head(10)

0    4.0
1    5.0
2    4.0
3    5.0
4    5.0
5    5.0
6    5.0
7    5.0
8    5.0
9    5.0
Name: overall, dtype: float64

In [73]:
# Creating product table
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS product (
    asin TEXT,
    imUrl TEXT,
    description TEXT,
    title TEXT,
    brand TEXT,
    price DECIMAL(10, 2) 
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully")
except Exception as error:
    print(f"Error: Could not create table. {error}")
finally:
    cursor.close()


Table created successfully


In [74]:
# Creating product_category table
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS product_category (
    asin TEXT,
    categories TEXT
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully")
except Exception as error:
    print(f"Error: Could not create table. {error}")
finally:
    cursor.close()

Table created successfully


In [104]:
# Creating product_review table
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS product_review (
    asin TEXT,
    reviewerID TEXT,
    reviewTime TEXT,
    summary TEXT,
    reviewText TEXT,
    overall DECIMAL(10,2)
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully")
except Exception as error:
    print(f"Error: Could not create table. {error}")
finally:
    cursor.close()

Table created successfully


In [103]:
connection.rollback()

In [76]:
# Creating product_review table
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS reviewer (
    reviewerID TEXT,
    reviewerName TEXT
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully")
except Exception as error:
    print(f"Error: Could not create table. {error}")
finally:
    cursor.close()

Table created successfully


In [63]:
connection.rollback()

In [100]:
import psycopg2.extras

def populate_tables(connection):
    cursor = connection.cursor()
    
    for _, row in amazon_merged_df.iterrows():
        # Insert into the 'product' table
        cursor.execute('INSERT INTO product (asin, imUrl, description, title, brand, price) VALUES (%s, %s, %s, %s, %s, %s)', 
                       (row['asin'], row['imUrl'], row['description'], row['title'], row['brand'], float(row['price'])))
        
        # Ensure the 'categories' column is a valid array format (1D array)
        categories = row['categories']
        
        # Handle categories if it's a string format like "[ 'Electronics', 'Home' ]"
        if isinstance(categories, str):
            # Remove unnecessary characters and split by comma
            categories = categories.strip("[]").replace("'", "").split(", ")
        
        # If categories is already a list but is multidimensional, flatten it
        if isinstance(categories, list) and any(isinstance(i, list) for i in categories):
            categories = [item for sublist in categories for item in sublist]
        
        # Insert categories into the product_category table as a 1D array
        cursor.execute('INSERT INTO product_category (asin, categories) VALUES (%s, %s)', (row['asin'], categories))

        
        # Insert into the 'product_review' table
        cursor.execute('INSERT INTO product_review (asin, reviewerID, reviewTime, summary, reviewText, overall) VALUES (%s, %s, %s, %s, %s, %s)',
                       (row['asin'], row['reviewerID'], row['reviewTime'], row['summary'], row['reviewText'], float(row['overall'])))
        
        # Insert into the 'reviewer' table
        cursor.execute('INSERT INTO reviewer (reviewerID, reviewerName) VALUES (%s, %s)', (row['reviewerID'], row['reviewerName']))
    
    connection.commit()
    cursor.close()


In [101]:
populate_tables(connection)

## Requirement 4
### Create code for the queries listed in Requirement_1 using the database you selected in Requirement_2. You are allowed to use gpt-3.5-turbo to assist you in the code generation.

#### All ChatGPT interactions for query recommendations are in a separate PDF file.
#### Queries that were entered into the model were prompted by: 
- I have Amazon reviews dataset. Write me a PostgreSQL query for the following: [query]

In [79]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    cursor = connection.cursor()
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';")
    for table in cursor.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    cursor = connection.cursor()
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_schema = 'public' AND table_name   = %s;", [table_name])
    for col in cursor.fetchall():
        column_names.append(col[0])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(connection):
        columns_names = get_column_names(connection, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [80]:
table_names = get_table_names(connection)
print(len(table_names), table_names)

4 ['product', 'product_category', 'product_review', 'reviewer']


In [81]:
column_names_product_table = get_column_names(connection, 'product')
print(len(column_names_product_table), column_names_product_table)

6 ['asin', 'imurl', 'description', 'title', 'brand', 'price']


In [82]:
get_database_info(connection)

[{'table_name': 'product',
  'column_names': ['asin', 'imurl', 'description', 'title', 'brand', 'price']},
 {'table_name': 'product_category', 'column_names': ['asin', 'categories']},
 {'table_name': 'product_review',
  'column_names': ['asin',
   'reviewerid',
   'reviewtime',
   'summary',
   'reviewtext',
   'overall']},
 {'table_name': 'reviewer', 'column_names': ['reviewerid', 'reviewername']}]

In [83]:
database_schema_dict = get_database_info(connection)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

print(database_schema_string)

Table: product
Columns: asin, imurl, description, title, brand, price
Table: product_category
Columns: asin, categories
Table: product_review
Columns: asin, reviewerid, reviewtime, summary, reviewtext, overall
Table: reviewer
Columns: reviewerid, reviewername


In [84]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about Amazon product reviews dataset. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

In [85]:
def ask_database(connection, query):
    """Function to query Postgres database with a provided SQL query."""
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        results = str(cursor.fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(connection, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

#### Query 1
#### How many products are there in every product subcategory?

In [86]:
cursor = connection.cursor()

query = '''
SELECT 
    subcategory,
    COUNT(DISTINCT asin) AS product_count
FROM 
    product_category,
    LATERAL unnest(string_to_array(categories, ',')) AS subcategory
GROUP BY 
    subcategory
ORDER BY 
    product_count DESC;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query1 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query1.head()


Query ran successfully


,subcategory,product_count
0,{Appliances,11236
1,"""Parts & Accessories""",6063
2,"""Refrigerator Parts & Accessories""",1469
3,"""Parts & Accessories""}",1220
4,"""Water Filters""}",1030


#### Query 2
#### How many product subcategories are in the provided datasets? 

In [87]:
cursor = connection.cursor()

query = '''
SELECT 
    COUNT(DISTINCT subcategory) AS unique_subcategory_count
FROM 
    product_category,
    LATERAL unnest(string_to_array(categories, ',')) AS subcategory;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query2 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query2.head()

Query ran successfully


,unique_subcategory_count
0,126


#### Query 3
#### What is the average review rating of all reviews? 

In [88]:
cursor = connection.cursor()

query = '''
SELECT AVG(overall) AS average_rating 
FROM product_review;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query3 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query3.head()

Query ran successfully


,average_rating
0,4.0847200473257473


#### Query 4
#### How many unique reviewers?

In [89]:
cursor = connection.cursor()

query = '''
SELECT COUNT(DISTINCT reviewerid) AS unique_reviewers 
FROM reviewer;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query4 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query4.head()

Query ran successfully


,unique_reviewers
0,132002


#### Query 5
#### How many reviews for every product subcategory?

In [35]:
cursor = connection.cursor()

query = '''
SELECT pc.categories, COUNT(pr.asin) AS review_count
FROM product_category pc
JOIN product_review pr ON pc.asin = pr.asin
GROUP BY pc.categories;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query5 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query5.head()

Query ran successfully


,categories,review_count
0,"{""Tools & Home Improvement"",""Rough Plumbing"",""...",2754
1,"{Appliances,""Appliance Services""}",8253
2,"{Appliances,""Appliance Warranties""}",6237
3,"{Appliances,""Beer Keg Refrigerators""}",578079
4,"{Appliances,""Beverage Refrigerators""}",803214


#### Query 6
#### What is the average size (characters/words) of the review text/body?

In [36]:
cursor = connection.cursor()

query = '''
SELECT 
    AVG(LENGTH(reviewtext)) AS average_review_length_in_characters
FROM 
    product_review
WHERE 
    reviewtext IS NOT NULL
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query6 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query6.head()

Query ran successfully


,average_review_length_in_characters
0,3.0000000000000000


#### Query 7
#### How many reviews submitted every January for years: 2011, 2012, 2013, 2014?

In [37]:
cursor = connection.cursor()

query = '''
SELECT 
    EXTRACT(YEAR FROM reviewtime::DATE) AS review_year,
    COUNT(*) AS review_count
FROM 
    product_review
WHERE 
    EXTRACT(MONTH FROM reviewtime::DATE) = 1
    AND EXTRACT(YEAR FROM reviewtime::DATE) IN (2011, 2012, 2013, 2014)
GROUP BY 
    review_year
ORDER BY 
    review_year;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query7 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query7.head()

Query ran successfully


,review_year,review_count
0,2011,2643
1,2012,4929
2,2013,14946
3,2014,20241


#### Query 8
#### How many reviews submitted for every product subcategory every May for years: 2011, 2012, 2013, 2014?

In [38]:
cursor = connection.cursor()

query = '''
SELECT 
    EXTRACT(YEAR FROM reviewtime::DATE) AS review_year,
    subcategory, 
    COUNT(*) AS review_count
FROM 
    product_review
JOIN 
    product_category ON product_review.asin = product_category.asin
JOIN 
    LATERAL unnest(string_to_array(product_category.categories, ',')) AS subcategory ON true
WHERE 
    EXTRACT(MONTH FROM reviewtime::DATE) = 5 
    AND EXTRACT(YEAR FROM reviewtime::DATE) IN (2011, 2012, 2013, 2014)
GROUP BY 
    review_year, subcategory
ORDER BY 
    review_year, subcategory;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query8 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query8.head()

Query ran successfully


,review_year,subcategory,review_count
0,2011,"""All-in-One Combination Washers & Dryers""}",315
1,2011,"""Beer Keg Refrigerators""}",1881
2,2011,"""Beverage Refrigerators""}",1080
3,2011,"""Built-In Dishwashers""}",6975
4,2011,"""Built-in Thermometers""}",675


#### It should be noted this query does produce the review_count for each year specified, however I only show the first few rows due to extreme load time. This can be seen in PgAdmin4.

In [55]:
connection.rollback()

#### Query 9
#### What is the average price of every product subcategory that has an average overall rating that is greater than 3?

In [ ]:
cursor = connection.cursor()

query = '''
SELECT
    subcategory,
    AVG(p.price) AS average_price
FROM
    product_review 
JOIN
    product_category ON product_review.asin = product_category.asin
JOIN
    LATERAL unnest(string_to_array(pc.categories, ',')) AS subcategory ON true
WHERE
    AVG(product_review.overall) > 3
GROUP BY
    subcategory;
'''


try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query9 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query9.head()

#### Query 10 
#### What are the top 3 reviewed products names in every product subcategory? 

In [56]:
cursor = connection.cursor()

query = '''
SELECT
    subcategory, 
    COUNT(*) AS review_count
FROM 
    product_review
JOIN 
    product_category ON product_review.asin = product_category.asin
JOIN 
    LATERAL unnest(string_to_array(product_category.categories, ',')) AS subcategory ON true
GROUP BY 
    subcategory
ORDER BY 
    review_count, subcategory
LIMIT 3;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query10 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query10.head()

Query ran successfully


,subcategory,review_count
0,"""Specialty Dishwashers""}",18
1,"""Drop-In Ranges""}",72
2,"""Dance & Electronic""",81


#### Query 11
#### How many products have a rating of less than 3?

In [90]:
cursor = connection.cursor()

query = '''
SELECT COUNT(*)
FROM product p
WHERE p.asin IN (
    SELECT pr.asin
    FROM product_review pr
    WHERE overall < 3
);
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query11 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query11.head()

Query ran successfully


,count
0,125157


#### Query 12
#### What is the average number of helpful votes on each review?

In [60]:
cursor = connection.cursor()

query = '''
SELECT
    pr.asin,
    AVG(helpful_votes) AS average_helpful_votes
FROM
    product_review pr
GROUP BY
    pr.asin
ORDER BY
    average_helpful_votes DESC;
;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query12 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query12.head()

Error: Could not create query. column "helpful_votes" does not exist
LINE 4:     AVG(helpful_votes) AS average_helpful_votes
                ^



TypeError: 'NoneType' object is not iterable

#### Query 13
#### What is the average price for every product with a perfect rating of 5?

In [102]:
cursor = connection.cursor()

query = '''
SELECT
    p.asin, 
    ROUND(AVG(p.price)::numeric, 2) AS average_price
FROM
    product_review pr
JOIN
    product p ON pr.asin = p.asin
WHERE
    pr.overall = 5 
GROUP BY
    p.asin;
'''

try:
    cursor.execute(query)
    connection.commit()
    print("Query ran successfully")
    data = cursor.fetchall()
except Exception as error:
    print(f"Error: Could not create query. {error}")
finally:
    cursor.close()

# Fetch all the rows returned by the executed query
# Retrieve column names 
colnames = [desc[0] for desc in cursor.description]

# Create a DataFrame using the fetched data and the column names.
query13 = pd.DataFrame(data, columns=colnames)

# Display the first five rows.
query13.head()

Query ran successfully


,asin,average_price
0,7301113188,NaN
1,B00002NATH,9.13
2,B00004SQH5,7.38
3,B00004SQHD,18.03
4,B00004SQHH,14.94
